## Note
- Table style can be updated in `assets/css/beautifuljekyll.css`

# Generator 

In [1]:
import pandas as pd
import re

# read csv file
# pubs = pd.read_csv("publications.csv", encoding = "ISO-8859-1")
pubs = pd.read_excel("publications.xlsx")
# pubs.info()

# generate md file
import os

md = "---\n"
md += "layout: page\n"
md += "title: \"Publications\"\n"
md += "---\n"

def process_authors(authors):
#     author_list = [re.sub(r'[^\x00-\x7F]+',' ', x).strip(' ') for x in authors.split(',')] # strip and strip    
    author_list = [x.strip(' ') for x in authors.split(',')] # strip and strip    
    s = ''
    for i in range(len(author_list)):
        author = author_list[i].replace('Tim Chen', '**Tim Chen**')        
#         if(author == 'Tim Chen'):
#             author = '**'+author+'**'
        s += author            
        
        if(i != len(author_list) - 1):
            s += ', '        
        else:
            s += '<br>'
    return s

# generate html
md += '''
<table>
<colgroup>
<col width="30%" />
<col width="70%" />
</colgroup>
<tbody>
'''

for idx, row in pubs.iterrows():
    md += '<tr>\n'
    md += f'<td><img src="/assets/img/{row["IMG"]}" width="250"></td>'
    md += f'<td markdown="span">'
    md += f"**{row['TITLE']}**<br><br>"
    md += process_authors(row['AUTHORS'])
    md += f"*{row['VENUE']}* {row['YEAR']}<br>"
    paper_name = row['PAPER_NAME']
    if pd.notna(paper_name):
#         md += f"\[[paper](/assets/publications/{row['PAPER_NAME']})\]"
        md += f'<a href="/assets/publications/{row["PAPER_NAME"]}" target="_blank">[paper]</a>'
    else:
        md += f"[paper]"
    vid_name = row['YOUTUBE']
    md += '  '
    if pd.notna(vid_name):
#         md += f"\[[youtube]({vid_name})\]"
        md += f'<a href="{vid_name}" target="_blank">[youtube]</a>'
    else:
        md += " "

    md += '</td>\n'
    md += '</tr>\n'
md+= '''
</tbody>
</table>
'''

with open("../pubs.md", 'w', encoding='utf8') as f:
# with open("./test.md", 'w', encoding='utf8') as f:
    f.write(md)
print('done')

# pure markdown version below, cannot control image width...
# md += '''
# |                  |                    |
# |----              |----                |
# '''
# for idx, row in pubs.iterrows():
#     md += '|'
# #     /assets/img/profile.jpg
# #     md += f"![alt text](../_publications/{row['IMG']})"
#     md += f'<img src="/assets/img/{row["IMG"]}" width="300">'
#     md += '|'
#     md += f"**{row['TITLE']}**<br><br>"
#     md += process_authors(row['AUTHORS'])
#     md += f"*{row['VENUE']}* {row['YEAR']}<br>"
#     md += '|\n'
# md += '|======================|    |\n'


done


In [21]:
a = " bcd "
re.sub(r'[^\x00-\x7F]+',' ', a)
print(a)
print (a)
a = a.strip()
print(a)

 bcd 
 bcd 
bcd


In [16]:
import regex
c = ' Tim Chen'
c.replace(/[^\x00-\x7F]/g,"");
print(c)
c = c.strip(' ')
print(c)

SyntaxError: invalid syntax (<ipython-input-16-5f4360a3af1e>, line 3)